In [ ]:
import numpy as np
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from scipy.special import comb
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, Session
import time as systime
import os

In [ ]:
# Definition of batches, the dimension d of each subsystem A and B, the frequency w, the number of partitions p and shots

batches = 3
d = 4
w = 0.1
p = 30
nshots = 8192
q = 2 * int(np.ceil(np.log(d) / np.log(2)))
num_qubits = 2*q + 1

In [ ]:
# Create a list to store job IDs and corresponding time values
job_info_hadamd4_qiskit_aachen = {}
# Create an array of time values
t_values = np.linspace(0, np.pi / w, p)

In [ ]:
# Initialize the analytical number of non-null Walsh angles and the angles/indices arrays
num_angles = int(q**2/4 + q)
a_j = np.zeros(num_angles, dtype=int)
indices = np.zeros(num_angles, dtype=int)

# Pre-calculate constants for angle computations
result_h1 = -(8**(q/2))*(1+2**(q/2))/8
result_h2 = (4**q)/16

# Calculate angles and indices
for j1 in range(int(q/2)):
    base_index = j1 + (j1+1)*int(q/2)
    
    a_j[j1] = result_h1/(2**j1)
    a_j[base_index] = a_j[j1]
    
    indices[j1] = 2**j1
    indices[base_index] = 2**(j1 + int(q/2))

    for j2 in range(int(q/2)):
        offset_index = base_index + j2 + 1
        a_j[offset_index] = result_h2/(2**(j2+j1))
        indices[offset_index] = indices[base_index] + 2**j2

In [ ]:
# This cell is complementary to the previous cell. Here we apply for both the original n qubit system and its copy all the cnots and z-rotations 
# This will result in the desired evolved state.

def gates(QC,position,c,t):
    binary_str = format(indices[position], f'0{q}b')[::-1]    
    msb_index = binary_str.rfind('1')

    #Applies the cnots (to the left of the z-rotation gate) for current a_j
    for i in range(q):
        if binary_str[i] == '1':
            if i != msb_index:
                QC.cx(i + 1 + c*q, msb_index + 1 + c*q)
    
    # Applies rotation gate to the current qubit
    theta_j = 2*a_j[position]*w*t/(2**q)
    QC.rz(theta_j, msb_index + 1 + c*q)
    
    # Applies cnots in reverse order
    for i in reversed(range(q)):
        if binary_str[i] == '1':
            if i != msb_index:
                QC.cx(i + 1 + c*q, msb_index + 1 + c*q)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit import QuantumCircuit, transpile
import time as systime
import os
import numpy as np

# === Configurações ===
service = QiskitRuntimeService()
backend_name = "ibm_aachen"
backend = service.backend(backend_name)
sampler = SamplerV2(backend=backend)
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"

job_info_path = "job_info_hadamd4_qiskit_aachen.npy"

# Carrega ou inicializa o dicionário
job_info_hadamd4_qiskit_aachen = (
    np.load(job_info_path, allow_pickle=True).item()
    if os.path.exists(job_info_path) else {}
)

# Containers
results = {}
job_ids = {}

# === Envio de jobs pendentes ===
for t in t_values:
    results[t] = []
    job_ids[t] = []

    # Descobre quais batches já existem para esse t
    submitted_batches = {batch for (tt, batch) in job_info_hadamd4_qiskit_aachen if np.isclose(tt, t)}

    print(f"\n🌀 Submitting jobs for t = {t:.6f} (já existem {len(submitted_batches)}/{batches} batches)")

    for batch in range(batches):
        if batch in submitted_batches:
            print(f"  ⚠️ Batch {batch} já existe para t = {t}, pulando...")
            continue

        # === Construção do circuito ===
        qc = QuantumCircuit(num_qubits, 1)
        for j in range(1,2*q + 1):
            qc.h(j)
        for c in range(2):
            for position in range(num_angles):
                gates(qc, position, c, t)
        qc.h(0)
        for i in range(1, int(q / 2) + 1):
            qc.cswap(0, i, i + q)
        qc.h(0)
        qc.measure(0, 0)

        transpiled = transpile(qc, backend=backend, optimization_level=3)

        # === Timer ===
        batch_start = systime.time()

        # Envio do job com tentativas
        while True:
            try:
                job = sampler.run([transpiled], shots=nshots)
                jid = job.job_id()
                job_ids[t].append(jid)
                job_info_hadamd4_qiskit_aachen[(t, batch)] = jid
                print(f"  ✅ Enviado t={t:.6f}, batch={batch}, job ID: {jid}")
                break
            except Exception as e:
                print(f"⚠️ Retry necessário (batch={batch}, t={t}): {e}")
                systime.sleep(30)

        # Monitoramento ao vivo
        wait_start = systime.time()
        while job.status() not in ["DONE", "CANCELLED", "ERROR"]:
            elapsed = systime.time() - wait_start
            print(f"    ⏳ Status: {job.status()}, aguardando... {elapsed:.1f}s ({elapsed / 60:.2f} min)")
            systime.sleep(10)

        # Tempo total
        batch_end = systime.time()
        duration = batch_end - batch_start

        # Salva tempo no log
        with open("batch_times_log.txt", "a") as f:
            f.write(f"t = {t}, batch = {batch}, job_id = {jid}, duration = {duration:.2f} sec ({duration / 60:.2f} min)\n")

        # Tenta pegar resultados
        if job.status() == "DONE":
            try:
                result = job.result()
                counts = result[0].join_data().get_counts()
                results[t].append(counts)
                print(f"    ✅ Feito. Counts: {counts}")
            except Exception as e:
                print(f"    ❌ Falha ao obter resultado: {e}")
        else:
            print(f"    ❌ Job falhou com status: {job.status()}")

# Salva dicionário final
np.save(job_info_path, job_info_hadamd4_qiskit_aachen)
print("\n✅ Todos os jobs pendentes foram processados e o arquivo .npy foi atualizado.")


In [ ]:
# CELL TO PROCESS INCOMPLETE DATA, IT SAVES THE DATA ITSELF
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService

# Load job ID dictionary
job_info = np.load("job_info_hadamd4_qiskit_aachen.npy", allow_pickle=True).item()
service = QiskitRuntimeService()


# Extract t_values from the job_info keys only
available_t_values = sorted(set(t for (t, _) in job_info.keys()))

# Prepare result containers
Tr_val = []
Tr_val_errors = []
all_trace = {t: [] for t in available_t_values}

cont = 0
for (t, batch), job_id in job_info.items():
    try:
        job = service.job(job_id)
        status = job.status()

        if status != "DONE":
            print(f"⏳ Skipping job {job_id} — status is {status}")
            continue

        result = job.result()
        counts = result[0].join_data().get_counts()
        p0 = sum(v for k, v in counts.items() if k[::-1][0] == '0') / nshots
        Tr = 2 * p0 - 1
        all_trace[t].append(Tr)

    except Exception as e:
        print(f"⚠️ Failed job {job_id} for t = {t}, batch = {batch}: {e}")
    cont += 1
    print(cont)

# Compute mean and stderr
for t in available_t_values:
    traces = all_trace[t]
    if len(traces) > 0:
        mean_trace = np.mean(traces)
        std_err = np.std(traces, ddof=1) / np.sqrt(len(traces))
        Tr_val.append(mean_trace)
        Tr_val_errors.append(std_err)
        print(f"📈 Time {t:.4f}: Mean Trace = {mean_trace:.6f}, StdErr = {std_err:.6f}")
    else:
        print(f"❌ No successful batches for t = {t}")

t_val = np.array(available_t_values)
# Save in NPZ format (structured and compact)
np.savez("results_hadamd4_qiskit_aachen.npz", t=t_val, Tr=np.array(Tr_val), err=np.array(Tr_val_errors))

# Salvar em CSV com cabeçalho
data_to_save = np.column_stack((t_val, Tr_val, Tr_val_errors))
np.savetxt("results_hadamd4_qiskit_aachen.csv", data_to_save,
           delimiter=",", header="t,Tr,err", comments="")


In [ ]:
# Calculate the divisors of the integer n
def get_all_divisors(n):
    # Initialize a list to store all divisors
    divisors = set()
    
    # Loop to find divisors
    for i in range(1, int(n ** 0.5) + 1):
        if n % i == 0:
            # If i is a divisor, n // i is also a divisor
            divisors.add(i)
            divisors.add(n // i)
    
    # Return the divisors in ascending order
    return sorted(divisors)

# Functions for calculating analytical Fourier modes and reduced purity
def results(n):
    # Auxiliary function for Fourier mode calculations
    divisors = get_all_divisors(n)
    z = len(divisors) - 1
    R = sum(divisors)
    z_red = int(np.ceil(z/2))
    s = 0
    zt = z + 1
    for i in range(z_red, z + 1):
        if divisors[i] >= 2**(q/2):
            if divisors[i] != divisors[z - i]:
                s += divisors[i] + divisors[z - i]
                zt -= 2
            else:
                s += divisors[i]
                zt -= 1
    return R - s, zt

def calc_alpha_n_analyt(n):
    # Analytical calculation of Fourier modes
    Rt, Zt = results(n)
    if n <= (2**(q/2)-1)*(2**(q/2)-1):
        return 4 / (4**q) * (Zt * (2**q + n) - (2**(q/2 + 1)) * Rt)
    else:
        return 0

def Trace_analytical(t):
    # Analytical calculation of the trace/reduced purity
    Tr_part = sum(calc_alpha_n_analyt(v) * np.cos(v * w * t) for v in range(1, int((2**(q/2) - 1)**2 + 1)))
    Tr_analit = ((2 * 8**(q/2) - 2**q) / 4**q) + Tr_part
    return Tr_analit

In [ ]:
# CÉLULA QUE OTIMIZA O FATOR DE CORREÇÃO, INDEPENDENTE DA CÉLULA DE PLOT ORIGINAL
from scipy.optimize import minimize_scalar

# Carrega os dados do CSV
t_val, Tr_val_exp, Tr_val_errors_exp = np.loadtxt("results_hadamd4_qiskit_aachen.csv",
                                             delimiter=",",
                                             skiprows=1,
                                             unpack=True)

# === Cópias seguras ===
n_val = np.arange(2, 2*(d-1)+1)

Tr_val_analyt = np.array(Trace_analytical(t_val))

# === Funções para αₙ e erro ===
def calc_alpha_n_exp(n, Tr_vals, Tr_errs):
    integrando = (2*w/np.pi) * Tr_vals * np.cos(n * w * t_val)
    errors_integrando = (2*w/np.pi) * Tr_errs * np.abs(np.cos(n * w * t_val))
    result = simpson(integrando, x=t_val)
    delta_t = np.pi/(w*(p-1))
    error_result = np.sqrt(np.sum((delta_t * errors_integrando)**2))
    return result, error_result

# === Limite inferior analítico B_n
def analytical_bound(n):
    n = np.asarray(n)
    cond = (n >= 2) & (n < 2**(q/2))
    result = np.zeros_like(n, dtype=float)
    result[cond] = (8 / 2**(2*q)) * (-(2**(q/2) - 1)*n[cond] + 2**(q/2)*(2**(q/2) - 1))
    return result


# Region 2 : Below alpha_p2 in [2, 2(d-1)] → "possibly prime"
def alpha_p2_func(x): # Renamed to avoid conflict with variable name
    base = (4/(2**(q/2))**4)*((2**(q/2))**2 + x - 2 * (2**(q/2)) * np.sqrt(x))
    return np.where(x < d, base + analytical_bound(x), base)
alpha_p2 = alpha_p2_func(n_val) # Applied the function to n_val

# === αₙ corrigidos para um fator arbitrário (só valores)
def calc_alpha_scaled_exp(scale_factor, Tr_vals, Tr_errs):
    scaled_vals = scale_factor * np.copy(Tr_vals)
    return np.array([calc_alpha_n_exp(n, scaled_vals, Tr_errs)[0] for n in n_val])

# === Função objetivo: soma das diferenças absolutas
def objective_exp(scale_factor, Tr_vals, Tr_errs, alpha_analytical):
    alpha_scaled = calc_alpha_scaled_exp(scale_factor, Tr_vals, Tr_errs)
    return np.sum(np.abs(alpha_scaled - alpha_analytical))

# === Modos analíticos
alpha_n_analyt = np.array([calc_alpha_n_analyt(n) for n in n_val])

# === Otimização
optimize_result_exp = minimize_scalar(
    objective_exp,
    bounds=(0.1, 3.0),
    method='bounded',
    args=(Tr_val_exp, Tr_val_errors_exp, alpha_n_analyt)
)

optimal_scale_factor_exp = optimize_result_exp.x
print(f"🔵 Melhor fator de escala experimental encontrado: {optimal_scale_factor_exp:.6f}")

# === Recalcular modos corrigidos (usando erro original)
Tr_val_exp_corrected = optimal_scale_factor_exp * Tr_val_exp
Tr_val_errors_exp_corrected = optimal_scale_factor_exp*Tr_val_errors_exp 
alpha_n_exp_corrected, alpha_n_errors_exp_corrected = map(np.array, zip(*[
    calc_alpha_n_exp(n, Tr_val_exp_corrected, Tr_val_errors_exp_corrected) for n in n_val
]))

# === Modos originais e simulados
alpha_n_exp_original, alpha_n_errors_exp_original = map(np.array, zip(*[
    calc_alpha_n_exp(n, Tr_val_exp, Tr_val_errors_exp) for n in n_val
]))
bound_n = [analytical_bound(n) for n in n_val]

# === Gráfico 1: pureza vs tempo
plt.figure()
plt.plot(t_val, Tr_val_analyt, label='Analytical', color='red')
plt.errorbar(t_val, Tr_val_exp, yerr=Tr_val_errors_exp, fmt='-o', capsize=5,
             label='Demonstration (Original)', color='blue', markersize=5)
plt.errorbar(t_val, Tr_val_exp_corrected, yerr=Tr_val_errors_exp_corrected, fmt='-o', capsize=5,
             label='Demonstration (Mitigated)', color='darkorange', markersize=5)
plt.xlabel('t', fontsize=16)
plt.ylabel(r'$\gamma_A(t)$', fontsize=16)
plt.title(f'Purity vs time for $d = {d}$', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)
plt.legend(fontsize=9.5)
plt.tight_layout()
plt.savefig("purity_vs_time_d4.pdf", dpi=300, bbox_inches='tight')
plt.show()


# === Gráfico 2: modos de Fourier
plt.figure()

# Plotting commands
plt.plot(n_val, alpha_n_analyt, marker='o', linestyle='-', label='Analytical', color='red')
plt.errorbar(n_val, alpha_n_exp_original, yerr=alpha_n_errors_exp_original, fmt='o', linestyle='-', capsize=5,
             label='Demonstration (Original)', color='blue')
plt.errorbar(n_val, alpha_n_exp_corrected, yerr=alpha_n_errors_exp_corrected, fmt='o', linestyle='-', capsize=5,
             label='Demonstration (Mitigated)', color='darkorange')
plt.plot(n_val, bound_n, label=r'Lower bound $B_n$', color='green')
plt.plot(n_val, alpha_p2, linewidth=2.2, label=r'Composite bound $P_n$', color='purple')
end = int((d - 1)**2)
y_alpha_p2_full = alpha_p2[(n_val >= 2) & (n_val <= end)]
x_full = n_val[(n_val >= 2) & (n_val <= end)]

# Create a proxy artist for the fill_between legend
from matplotlib.patches import Patch
blue_patch = Patch(color='cornflowerblue', alpha=0.7, label='Possibly prime')
plt.fill_between(x_full, bound_n, y_alpha_p2_full, color='cornflowerblue', alpha=0.7)


# Fix legend order manually
handles, labels = plt.gca().get_legend_handles_labels()

# Add the proxy artist to handles and its label to labels
handles.append(blue_patch)
labels.append(blue_patch.get_label()) # This line adds the label for the patch

desired_order = [
    labels.index('Analytical'),
    labels.index('Demonstration (Original)'),
    labels.index('Demonstration (Mitigated)'),
    labels.index(r'Lower bound $B_n$'),
    labels.index(r'Composite bound $P_n$'), # Corrected: Added missing closing single quote
    labels.index('Possibly prime') # Ensure this label is in the desired order
]

# Reorder handles and labels based on the desired_order
handles = [handles[idx] for idx in desired_order]
labels = [labels[idx] for idx in desired_order]


plt.xlabel('n', fontsize=16)
plt.ylabel(r'$\alpha_n$', fontsize=16)
plt.title(f'Fourier modes for $d = {d}$', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(handles=handles, labels=labels, fontsize=9.5)
plt.grid(True)
plt.tight_layout()
plt.savefig("fourier_modes_d4.pdf", dpi=300, bbox_inches='tight')
plt.show()
